In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train = x_train/255
x_test = x_test/255
y_cat_train = to_categorical(y_train,10)
y_cat_test = to_categorical(y_test,10)

In [ ]:
### Function to improve  baseline model

def build_model (hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    
    model.add(layers.Dense(units=512,
                          activation='relu'))
    
    model.add(layers.Dense(units=384,
                          activation='relu'))
    # Adding dropout
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.2))
    
    # Output layer
    model.add(layers.Dense(10,activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
    # Testing different learning rates
        hp.Choice('learning_rate',
                 values=[1e-2, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
model = build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = RandomSearch(
    hypermodel = build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir/colab2',
    project_name='ANN tune dropout and lr')

INFO:tensorflow:Reloading Oracle from existing project my_dir/colab2/ANN tune dropout and lr/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/colab2/ANN tune dropout and lr/tuner0.json


In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
dropout (Boolean)
{'default': False, 'conditions': []}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_train, y_cat_train,
            epochs=20,
            validation_data=(x_test, y_cat_test),
            callbacks=[early_stop, [keras.callbacks.TensorBoard("/my_dir/colab2")]])

INFO:tensorflow:Oracle triggered exit


In [ ]:
# comparing all best results obtained
tuner.results_summary()

Results summary
Results in my_dir/colab2/ANN tune dropout and lr
Showing 10 best trials
Trial summary
Hyperparameters:
dropout: False
learning_rate: 0.0001
Score: 0.5231000185012817
Trial summary
Hyperparameters:
dropout: True
learning_rate: 0.0001
Score: 0.49900001287460327
Trial summary
Hyperparameters:
dropout: False
learning_rate: 0.01
Score: 0.17890000343322754
Trial summary
Hyperparameters:
dropout: True
learning_rate: 0.01
Score: 0.10000000149011612
